In [1]:
import math
from interval import interval, imath, fpu

In [2]:
class Poly:
    def __init__(self, coef:list, order:int):
        self.order=order
        if len(coef)<(order+1):
            for i in range(order+1-len(coef)):
                coef.append(0) 
        self.coef=coef

    def __repr__(self):
        return 'Poly(%s, %d)'%(self.coef, self.order)
   
    def fixshape(x, y):
        order=max(x.order,y.order)
        x1=x.coef[:]
        y1=y.coef[:]
        if x.order<order:
            for i in range(order-x.order):
                x1.append(0)
        else:
            for i in range(order-y.order):
                y1.append(0)
        return Poly(x1,order), Poly(y1,order), order
        
    def fixshape1(x, y):
        order=x.order+y.order
        x1=x.coef[:]
        y1=y.coef[:]
        for i in range(order-x.order):
            x1.append(0)
        for j in range(order-y.order):
            y1.append(0)
        return Poly(x1,order), Poly(y1,order), order   
    
    def firstnonzero(self):
        order=self.order
        nonzero=order+1
        for i in range(order+1):
            if self.coef[i]!=0:
                nonzero=i
                break
        return nonzero
    
    def __add__(self, other):
        coef=[]
        if isinstance(other,(int,float)):
            other=Poly([other],0)
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]+other1.coef[i])
        return Poly(coef,order)
    
    def __radd__(self,other):
        return self+other
    
    def __sub__(self, other):
        coef=[]
        if isinstance(other,(int,float)):
            other=Poly([other],0)
        self1, other1, order=self.fixshape(other)
        for i in range(order+1):
            coef.append(self1.coef[i]-other1.coef[i])
        return Poly(coef,order)
    
    def __rsub__(self,other):
        if isinstance(other,(int,float)):
            other=Poly([other],0)
        return other-self
    
    def __mul__(self, other):
        coef=[]
        if isinstance(other,(int,float)):
            if other==0:
                return 0
            else:
                other=Poly([other],0)
        self1, other1, order=Poly.fixshape1(self, other)
        coef.append(self1.coef[0]*other1.coef[0])
        for i in range(1,order+1):
            s=0
            for j in range(i+1):
                s+=self1.coef[j]*other1.coef[i-j]
            coef.append(s)
        return Poly(coef,order)
    
    def __rmul__(self,other):
        return self*other
    
    def __truediv__(self, other):
        if isinstance(other,(int,float)):
            if other==0:
                raise Exception("Cannot dividied by zero!")
            else:
                coef=[]
                for i in range(self.order+1):
                    coef.append(self.coef[i]/other)
                return Poly(coef, self.order)
        else:
            raise Exception("Can only do scalar division!")

    def zero(n):
        coef=[]
        coef.append(0)
        for i in range(n):
            coef.append(0)
        return Poly(coef,n)
    
    def one(n):
        coef=[]
        coef.append(1)
        for i in range(n):
            coef.append(0)
        return Poly(coef,n)
    
    def __eq__(self,other):
        self1, other1, order=Poly.fixshape(self, other)
        return self1.coef==other1.coef
    
    def square(self):
        order=self.order*2
        for a in range(self.order):
            self.coef.append(0)
        coef=[]
        coef.append((self.coef[0])**2)
        for i in range(1,order+1):
            coef.append(self.squarecoef(i,self.coef))
        return Poly(coef,order)        
    
    def squarecoef(self,i,coef):
        s=0
        iend=int((i-2+i%2)/2)
        for j in range(iend+1):
            s+=coef[j]*coef[i-j]
        s=2*s
        if i%2==0:  
            s+=coef[int(i/2)]**2
        return s
    
    def sqrt(self):
        order=self.order
        l0nz=self.firstnonzero()
        if l0nz%2==1:
            raise ValueError("First non-vanishing Taylor coefficient must be an EVEN POWER to expand SQRT around 0.\n")
        lnull=int(l0nz/2)
        aux=math.sqrt(self.coef[l0nz])
        coef=Poly.zero(order).coef
        coef[lnull]=aux
        for i in range(lnull+1,order-l0nz+1):
            coef[i]=self.sqrtcoef(i,self.coef,coef,lnull)
        return Poly(coef,order)

    def sqrtcoef(self,i,ac,coef,lnull):
        s=0
        kodd=(i-lnull)%2
        kend=int((i-lnull-2+kodd)/2)
        for j in range(lnull+1,lnull+kend+1):
            s+=coef[j]*coef[i+lnull-j]
        aux=ac[i+lnull]-2*s
        if kodd==0:
            aux=aux-(coef[kend+lnull+1])**2
        s=aux/(2*coef[lnull])
        return s
    
    def __pow__(self,n):
        if type(n)==int:
            order=self.order*n
            for a in range(self.order):
                self.coef.append(0)
            if n<0:
                raise Exception("n must ≥ 0")
            elif n==0:
                return Poly.one(order)
            elif n%2==0:
                if n==2:
                    return self.square()
                else:
                    p=int(n/2)
                    return Poly(self.__pow__(p).square().coef,order)
            elif n==1:
                return self
            else:
                p=int((n-1)/2)
                return Poly((self*self.__pow__(p).square()).coef,order)
        elif type(n)==float:
            order=self.order
            uno=Taylor.one(order)
            if n==0:
                return uno
            elif n==0.5:
                return self.sqrt()
            l0nz=self.firstnonzero()
            lnull=n*l0nz
            lnull=int(lnull)
            aux=(self.coef[l0nz])**n
            coef=Taylor.zero(order).coef
            coef[lnull]=aux
            k0=lnull+l0nz
            for i in range(k0+1,order+1):
                coef[i-l0nz]=self.powcoef(i,self.coef,n,coef,l0nz)
            return Poly(coef,order)

    def powcoef(self,i,ac,x,coef,l0nz):
        s=0
        for j in range(i-l0nz):
            aux=x*(i-j)-j
            s+=aux*ac[i-j]*coef[j]
        aux=i-l0nz*(x+1)
        s=s/(aux*ac[l0nz])
        return s
    
    def exp(self):
        order=self.order
        coef=[]
        coef.append(math.exp(self.coef[0]))
        for i in range(1,order+1):
            s=0
            for j in range(i):
                s+=(i-j)*self.coef[i-j]*coef[j]
            coef.append(s/i)
        return Poly(coef,order)
    
    
    def log(self):
        order=self.order
        coef=[]
        coef.append(math.log(self.coef[0]))
        for i in range(1,order+1):
            s=0
            for j in range(1,i):
                s+=(i-j)*self.coef[j]*coef[i-j]
            coef.append((self.coef[i]-s/i)/self.coef[0])
        return Poly(coef,order)
    
    def sincos(self):
        order=self.order
        scoef=[];ccoef=[]
        scoef.append(math.sin(self.coef[0]))
        ccoef.append(math.cos(self.coef[0]))
        for i in range(1,order+1):
            s=0;c=0
            for j in range(1,i+1):
                s+=j*self.coef[j]*ccoef[i-j]
                c-=j*self.coef[j]*scoef[i-j]
            scoef.append(s/i)
            ccoef.append(c/i)
        return Poly(scoef,order), Poly(ccoef,order)
    
    def sin(self):
        return self.sincos()[0]
    
    def cos(self):
        return self.sincos()[1]
    
    def tan(self):
        order=self.order
        coef1=[]
        coef2=[]
        t=math.tan(self.coef[0])
        coef1.append(t)
        coef2.append(t**2)
        for i in range(1,order+1):
            coef1.append(self.tancoef(i,self.coef,coef2))
            coef2.append(self.squarecoef(i,coef1))
        return Poly(coef1,order)
    
    def tancoef(self,i,coef,coef2):
        s=0
        for j in range(i):
            s+=(i-j)*coef[i-j]*coef2[j]
        s=self.coef[i]+s/i
        return s
    
    def diffPoly(self):
        order=self.order
        coef=[]
        coef.append(self.coef[1])
        for i in range(2,order+1):
            coef.append(i*self.coef[i])
        coef.append(0)
        return Poly(coef,order)
        
    def intePoly(self,x):
        order=self.order
        coef=[]
        coef.append(x)
        for i in range(1,order+1):
            coef.append(self.coef[i-1]/i)
        return Poly(coef,order)
    
    #Horner's rule
    def __call__(self,dx):
        order=self.order
        s=self.coef[order]
        for i in range(order-1,-1,-1):
            s=s*dx+self.coef[i]
        return s
        
    def deriv(self,n):
        s=math.factorial(n)*self.coef[n]
        return s
    
    # f(g(x)),self=f,other=g
    def compose(self,other):
        return self(other)

    def truncation(self,n):
        return Poly(self.coef[:n+1],n)    
    
    def bound_naive(self,x):
        B=self.coef[0]
        n=self.order
        for i in range(1,n+1):
            B+=self.coef[i]*(x**i)
        return B
    
    def bound(self,x,c='best'):
        if c=='naive' or c=='n':
            return self.bound_naive(x)
        elif c=='Horner' or c=='H':
            return self(x)
        elif c=='best':
            return self(x)&self.bound_naive(x)
        else:
            raise Exception("The options mush be naive(n), Horner(H) or best")
    
    #subdivision k times
    def subdivision(self,x,k=100):
        if 0<k<=1000:
            sup=x[0].sup
            inf=x[0].inf
            l=(sup-inf)/k
            I=[]
            for i in range(k):
                I.append(interval([inf+i*l,inf+(i+1)*l]))
            B=self.bound(I[0])
            for j in range(1,k):
                B=B|self.bound(I[j])
            return B
        else:
            raise Exception("k must between 0~1000!")
    

In [3]:
a=Poly([0,1,0,0,0],4)
b=Poly([1,2,3],2)
c=Poly([0,1,2,3,4],4)
d=Poly([0,1,2],2)

In [4]:
(b*d).truncation(2)

Poly([0, 1, 4], 2)

In [5]:
a+b

Poly([1, 3, 3, 0, 0], 4)

In [6]:
(a+b)(1)

7

In [7]:
2+a

Poly([2, 1, 0, 0, 0], 4)

In [8]:
a+2

Poly([2, 1, 0, 0, 0], 4)

In [9]:
a-b

Poly([-1, -1, -3, 0, 0], 4)

In [10]:
a-2

Poly([-2, 1, 0, 0, 0], 4)

In [11]:
2-a

Poly([2, -1, 0, 0, 0], 4)

In [12]:
b*c

Poly([0, 1, 4, 10, 16, 17, 12], 6)

In [13]:
2*b

Poly([2, 4, 6], 2)

In [14]:
0*b

0

In [15]:
c/2

Poly([0.0, 0.5, 1.0, 1.5, 2.0], 4)

In [16]:
a*(b+1)

Poly([0, 2, 2, 3, 0, 0, 0], 6)

In [17]:
b.coef[0]+2*c+3*c**2

Poly([1, 2, 7, 18, 38, 60, 75, 72, 48], 8)

In [18]:
b.compose(c)

Poly([1, 2, 7, 18, 38, 60, 75, 72, 48], 8)

In [19]:
b(c)

Poly([1, 2, 7, 18, 38, 60, 75, 72, 48], 8)

In [20]:
(b*c).truncation(2)

Poly([0, 1, 4], 2)

In [21]:
(b*c).truncation(2)(1)

5

In [22]:
b(interval([-1,1]))

interval([-4.0, 6.0])

In [23]:
b.bound_naive(interval([-1,1]))

interval([-1.0, 6.0])

In [24]:
b.bound(interval([-1,1]))

interval([-1.0, 6.0])

In [25]:
b.bound(interval([-1,1]),'n')

interval([-1.0, 6.0])

In [26]:
b.bound(interval([-1,1]),'H')

interval([-4.0, 6.0])

In [27]:
b.bound(interval([-1,1]),'best')

interval([-1.0, 6.0])

In [28]:
b.subdivision(interval([-1,1]),100)

interval([0.6463999999999999, 6.0])

In [29]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,3])

interval([-9.0, 39.0])

In [30]:
f1=Poly([0,1,1],2)
f2=Poly([0,0,-1,1],3)
f3=f1+f2
f3(interval[0,3])

interval([0.0, 30.0])

In [31]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,3])

interval([-9.0, 39.0])

In [32]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,2])

interval([-4.0, 14.0])

In [33]:
(lambda x: x**2 + x + x**3 - x**2)(interval[2,3])

interval([5.0, 35.0])

In [34]:
(lambda x: x**2 + x + x**3 - x**2)(interval[0,2])|(lambda x: x**2 + x + x**3 - x**2)(interval[2,3])

interval([-4.0, 35.0])

In [35]:
a.exp()

Poly([1.0, 1.0, 0.5, 0.16666666666666666, 0.041666666666666664], 4)

In [36]:
(Poly.one(1)-a).log()

Poly([0.0, -1.0, -0.5, -0.3333333333333333, -0.25], 4)

In [37]:
a.sin()(1)

0.8333333333333334

In [38]:
a.cos()(1)

0.5416666666666667

In [39]:
a.tan()(1)

1.3333333333333333

In [40]:
k=0
if 0<k<10:
    a=1
else: a=0
a

0

In [17]:
try:   
    !jupyter nbconvert --to python Polynomials.ipynb
    # python即转化为.py，script即转化为.html
    # file_name.ipynb即当前module的文件名
except:
    pass   

[NbConvertApp] Converting notebook Polynomials.ipynb to python
[NbConvertApp] Writing 11479 bytes to Polynomials.py
